In [1]:
# module set
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common import exceptions

import time
import lxml

from collections import OrderedDict
from itertools import repeat

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#피파인벤 연결
driver = webdriver.Chrome('./chromedriver')
player_url = 'http://fifaonline4.inven.co.kr/dataninfo/player/?code='

In [3]:
seasons = [['LH','TKL','20KL','19S', '19UCL', '20TOTY','20TOTY-N'],
          ['NG', 'COC', 'OTW', '19A', '19', 'HOT', '19TOTS'],
          ['ICON', 'TC', '18S', 'MC-ICON', '18TOTY', 'GR', '19KFA'],
          ['18A', 'TT', 'TB', 'MCFC', '18KFA', 'NHD']]

In [4]:
def click_by_class(x, y):
    driver.get('http://fifaonline4.inven.co.kr/dataninfo/player/')
    driver.implicitly_wait(5)
#     //*[@id="fifaonline4Db"]/div[2]/ul[1]/li[2]/div[2]/div[2]/label[1]/span
    temp_path = f'//*[@id="fifaonline4Db"]/div[2]/ul[1]/li[2]/div[2]/div[{x}]/label[{y}]/span'
    button = driver.find_element_by_xpath(temp_path).click()
    driver.implicitly_wait(10)
    search = driver.find_element_by_xpath('//*[@id="fifaonline4Db"]/div[2]/ul[2]/li[17]/button[1]/img')
    search.click()
    driver.implicitly_wait(10)

In [5]:
def players_info(html):
    soup = bs(html, 'html.parser')
    search_url_list = []
    
    for link in soup.find_all('a'):
        if 'href' in link.attrs:
            li = link.attrs['href'].split('\n')
            if li[0].startswith(player_url):
                search_url_list.append(li[0])
            else:
                continue
    search_url_list = list(OrderedDict(zip(search_url_list, repeat(None))))
    print("num of url : ", len(search_url_list))
    print(search_url_list[:5])
    return search_url_list

In [6]:
def get_comment(urls):
    total_df = pd.DataFrame()
    for i in range(len(urls)):
        driver.get(urls[i])
        driver.implicitly_wait(2)

        html = driver.page_source
        one_soup = bs(html, 'html.parser')
        one_player_db=one_soup.find_all('span', class_='comment')
        one_player_name = one_soup.find('div', class_="fifa4 tooltip_title")
        player_name = one_player_name.text.strip()
        player_list = [one_player_db]
        try:
            while True: # 댓글 페이지가 몇개인지 모르므로.            
                element = driver.find_element_by_xpath('//*[@id="cmtPaging"]/span/span[3]/a')
                driver.execute_script("arguments[0].click();", element)
                driver.implicitly_wait(2)
                html = driver.page_source
                one_soup = bs(html, 'html.parser')
                comments = one_soup.find_all('span', class_='comment')
                player_list.append(comments)

        except exceptions.ElementNotVisibleException as e: # 페이지 끝
            pass

        except Exception as e: # 다른 예외 발생시 확인
            print(e)

        cmt_list = []
        for db in player_list:
            for cmt in db:
                cmt_list.append(cmt.text)
        one_df = pd.DataFrame({'name': player_name, 'comment': cmt_list})
        total_df = pd.concat([total_df, one_df], axis = 0)
    total_df = total_df.reset_index(drop=True)
    return total_df

In [ ]:
for i in range(0, 4):
    for j in range(7):
        if i==3 and j == 6:
            break
        click_by_class(i+1, j+1)
        time.sleep(10)
        html = driver.page_source
        urls = players_info(html)
        comment_df = get_comment(urls)
        comment_df.to_csv("./comment_data/"+seasons[i][j]+'_comment.csv', mode='w')
    

In [15]:
def get_comment_ex(urls):
    total_df = pd.DataFrame()
    for i in range(2):
        driver.get(urls[i])
        driver.implicitly_wait(2)

        html = driver.page_source
        one_soup = bs(html, 'html.parser')
        one_player_db=one_soup.find_all('span', class_='comment')
        one_player_name = one_soup.find('div', class_="fifa4 tooltip_title")
        player_name = one_player_name.text.strip()
        player_list = [one_player_db]
        try:
            while True: # 댓글 페이지가 몇개인지 모르므로.            
                element = driver.find_element_by_xpath('//*[@id="cmtPaging"]/span/span[3]/a')
                driver.execute_script("arguments[0].click();", element)
                driver.implicitly_wait(2)
                html = driver.page_source
                one_soup = bs(html, 'html.parser')
                comments = one_soup.find_all('span', class_='comment')
                player_list.append(comments)

        except exceptions.ElementNotVisibleException as e: # 페이지 끝
            pass

        except Exception as e: # 다른 예외 발생시 확인
            print(e)

        cmt_list = []
        for db in player_list:
            for cmt in db:
                cmt_list.append(cmt.text)
        one_df = pd.DataFrame({'name': player_name, 'comment': cmt_list})
        total_df = pd.concat([total_df, one_df], axis = 0)
    total_df = total_df.reset_index(drop=True)
    return total_df

In [ ]:
#apply each class
click_by_class(4, 7)
time.sleep(10)
html = driver.page_source
urls = players_info(html)
comment_df = get_comment(urls)
comment_df.to_csv("./comment_data/"+seasons[3][6]+'_comment.csv', mode='w')

In [ ]:
#check saved data
data = pd.read_csv('./comment_data/'+seasons[3][1]+"_comment.csv")
data